In [1]:
!python --version

Python 3.7.9


# 3.7.9 입장 방법 

1. anaconda prompt 진입 
2. activate py37 
3. jupyter notebook 

!pip install tensorflow

!pip list 

# 일반적인 선형회귀 가정 

In [4]:
import tensorflow as tf
import numpy as np

# 선형회귀를 가정한다
- np.sum((beta[0] + beta[1]*x - y)**2) array 타입의 경우 최소제곱법 추정
- np.mean((beta[0] + beta[1]*x - y)**2) MSE 처리시 
- 리스트로 처리할 경우 루프처리 

In [81]:
x = [1,2,3,4]
y = [0,-1,-3,-3]

# 1. 손실함수 정의 

In [82]:
def mse(beta,x,y):
    n = len(x)
    val = 0.0
    
    for i in range(n):
        # np.sum((beta[0] + beta[1]*x - y)**2) array 타입의 경우 최소제곱법 추정
        # np.mean((beta[0] + beta[1]*x - y)**2) MSE 처리시 
        # 리스트로 처리할 경우 루프처리 
        val += (beta[0] + beta[1]*x[i] - y[i])**2 / n
    return val

# 2. 기울기 함수도 가정해보자 
- val += (beta[0] + beta[1]* x[i] - y[i])**2 / n 를 미분해서
- val += 2 * error * np.array([x[i], 1])/n 만들었다

In [83]:
def grad_loss(beta,x,y):
    dim = len(beta)
    n = len(x)
    val = np.array([0.0,0.0])
    for i in range(n):
        error = beta[0] + beta[1]*x[i] - y[i]
        val += 2.0*error * np.array([1.0, x[i]]) / n
    return val

# 3. 최적화 과정(Gradient Descent)
- 기본적으로 0.01

In [132]:
maxIter = 1
learning_rate = 0.01
beta0 = np.array([0,0])

for i in range(maxIter):
    grad = grad_loss(beta0, x , y)
    beta1 = beta0 - learning_rate * grad
    beta0 = beta1 
    print(beta0, mse(beta0, x, y))

TypeError: mse() takes 1 positional argument but 3 were given

# 2. 텐서 플로우를 이용한 선형회귀

## 2.1 텐서플로우의 변수 
- Constant (상수)
- Variable (가변값)

### 초기값 지정 
- beta = tf.Variable([0,0], dtype=float64) 이렇게 지정할 수도 있음
- 위에서 지정한 x와 y값을 그대로 이용하겠다
- 은 아니고 constant 지정하겠다

In [101]:
x = [1,2,3,4]
y = [0,-1,-3,-3]

In [102]:
x = tf.constant([1,2,3,4], dtype=tf.float64)
y = tf.constant([0,-1,-3,-3], dtype=tf.float64)

In [103]:
a = tf.Variable([0], dtype=tf.float64) # 기울기 x에 대한, beta1, w1, 변수의 영향력(회귀계수)
b = tf.Variable([0], dtype=tf.float64) # y절편, beta0, bias(편향)

beta = tf.Variable([0,0])

# 2.2 손실함수와 최적화 함수 정의
- 리니어모델 , 손실함수

In [104]:
def linear_model(x):
    return b + a*x 


def mse(yhat, y):
    return tf.reduce_sum(tf.pow(yhat-y,2)) / ( x.shape[0]) #*2 
    # reduce_mean 하고 분수 없애기 
    
optimizer = tf.optimizers.SGD(0.01)

# 2.3 최적화 과정 정의 

In [105]:
def optimization():
    #  주어진 입력변수에 대해서 자동으로 기울기를 구해준다. 
    with tf.GradientTape() as grad: 
        yhat = linear_model(x)
        loss = mse(yhat,y)
        
        
    # 기울기 계산 
    gradients = grad.gradient( loss, [ a , b ] )
    
    # a,b 업데이트 
    optimizer.apply_gradients( zip( gradients, [ a, b ] ) )

- beta0 = b 
- beta1 = 기울기 

# 2.4 학습(training)

In [106]:
maxIter = 1
for i in range(maxIter):
    optimization()
    yhat = linear_model(x)
    loss = mse(yhat, y)
    print('step : {}, loss: {}, a: {}, b; {}'.format(i, loss, a, b))

step : 0, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.115])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.035])>


    optimization()
    yhat = linear_model(x)
    loss = mse(yhat, y)
    
    제외하고 최적화 과정을 투입해도 같은 결과가 도출됨

In [107]:
 maxIter = 10
for i in range(maxIter):
    with tf.GradientTape() as grad: 
        yhat = linear_model(x)
        loss = mse(yhat,y)
        
    # 기울기 계산 
    gradients = grad.gradient( loss, [ a , b ] )
    
    # a,b 업데이트 
    optimizer.apply_gradients( zip( gradients, [ a, b ] ) )
    print('step : {}, loss: {}, a: {}, b; {}'.format(i, loss, a, b))

step : 0, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.211])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.06355])>


# 원래 했던 대로 np.array로 계산한다면?

In [126]:
x = np.array([1,2,3,4])
y = np.array([0,-1,-3,-3])

In [127]:
beta0 = np.array([0,0])

# 2.2 손실함수와 최적화 함수 정의
- 리니어모델 , 손실함수

In [128]:
def mse(beta):
    yhat = beta[0] + beta[1]*x 
    
    return np.mean((yhat-y)**2)

# 텐서에서는 MSE와 리니어 모델로 분리 

# 2.3 최적화 과정 정의 

In [129]:
def optimization():
    #  주어진 입력변수에 대해서 자동으로 기울기를 구해준다. 
    with tf.GradientTape() as grad: 
        yhat = linear_model(x)
        loss = mse(yhat,y)
        
        
    # 기울기 계산 
    gradients = grad.gradient( loss, [ a , b ] )
    
    # a,b 업데이트 
    optimizer.apply_gradients( zip( gradients, [ a, b ] ) )

- beta0 = b 
- beta1 = 기울기 

# 2.4 학습(training)

    optimization()
    yhat = linear_model(x)
    loss = mse(yhat, y)
    
    제외하고 최적화 과정을 투입해도 같은 결과가 도출됨

In [133]:
maxIter = 10
for i in range(maxIter):
    # 기울기 함수를 통해 기울기 계산(loss , a, b)
    grad = grad_loss(beta0,x,y)
    beta1 = beta0 - learning_rate * grad 

    # a,b 업데이트 
    beta0 = beta1 
    
    # 출력 
    print('step : {}, loss: {}, a: {}, b; {}'.format(i, loss, a, b))

step : 0, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.211])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.06355])>
step : 1, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.211])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.06355])>
step : 2, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.211])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.06355])>
step : 3, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.211])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.06355])>
step : 4, loss: 3.425537526909821, a: <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.211])>, b; <tf.Variable 'Variable:0' shape=(1,) dtype=float64, numpy=array([-0.06355])>
step : 5, loss: 3.425537526909